In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, ReLU
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, StandardScaler
import os

# Load data
csv_path = os.path.join('..', 'Data', 'Diagested_data', 'Numeric_resume_data.csv')
data = pd.read_csv(csv_path)
df = pd.DataFrame(data)

# Define trait columns
trait_columns = [
    'Leadership', 'Communication', 'Teamwork', 'Problem Solving', 'Creativity',
    'Adaptability', 'Work Ethic', 'Time Management', 'Interpersonal Skills', 
    'Attention to Detail', 'Initiative', 'Analytical Thinking', 'Emotional Intelligence', 
    'Integrity', 'Resilience', 'Cultural Awareness', 'Programming Languages', 
    'Technical Skills', 'Office Tools'
]

# Convert job titles to numerical labels
job_titles = df['Job Title'].unique()
job_domains = df[['Job Title', 'Domain']].drop_duplicates().set_index('Job Title')['Domain'].to_dict()
job_title_to_index = {title: idx for idx, title in enumerate(job_titles)}
index_to_job_title = {idx: title for title, idx in job_title_to_index.items()}
df['Job Label'] = df['Job Title'].map(job_title_to_index)

# Prepare feature and target arrays
X = df[trait_columns].values
y = df['Job Label'].values

# Feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Augment data with Gaussian noise
def add_gaussian_noise(X, noise_factor=0.05):
    noise = np.random.normal(loc=0.0, scale=noise_factor, size=X.shape)
    X_noisy = X + noise
    return np.clip(X_noisy, 0., 1.)

X_augmented = add_gaussian_noise(X_scaled)

# Combine original and augmented data
X_combined = np.vstack((X_scaled, X_augmented))
y_combined = np.hstack((y, y))

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y_combined, test_size=0.2, random_state=42)

# Convert labels to one-hot encoding
num_classes = len(job_titles)
lb = LabelBinarizer()
lb.fit(y_combined)  # Fit on the entire set of labels to ensure it has all classes
y_train_onehot = lb.transform(y_train)
y_test_onehot = lb.transform(y_test)

# Define a neural network model
model = Sequential([
    Dense(128, input_shape=(X_train.shape[1],)),
    ReLU(),
    Dropout(0.5),
    Dense(64),
    ReLU(),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with augmented data
model.fit(X_train, y_train_onehot, epochs=1000, batch_size=64, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test_onehot)
print(f'Test Accuracy: {accuracy:.2f}')

# Define the save path for the model
model_save_path = os.path.join('..', 'Code', 'Pickles', 'job_recommendation_algorithm.h5')

# Ensure the directory exists
os.makedirs(os.path.dirname(model_save_path), exist_ok=True)

# Save the model to the specified file
model.save(model_save_path)


c:\Users\yehus\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\yehus\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\yehus\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Epoch 1/1000
33/33 [==============================] - 1s 10ms/step - loss: 7.3457 - accuracy: 4.7596e-04 - val_loss: 7.3483 - val_accuracy: 0.0019
Epoch 2/1000
33/33 [==============================] - 0s 5ms/step - loss: 7.2611 - accuracy: 0.0038 - val_loss: 7.3875 - val_accuracy: 0.0038
Epoch 3/1000
33/33 [==============================] - 0s 4ms/step - loss: 7.1231 - accuracy: 0.0038 - val_loss: 7.5655 - val_accuracy: 0.0038
Epoch 4/1000
33/33 [==============================] - 0s 5ms/step - loss: 6.9180 - accuracy: 0.0052 - val_loss: 7.8189 - val_accuracy: 0.0057
Epoch 5/1000
33/33 [==============================] - 0s 5ms/step - loss: 6.7113 - accuracy: 0.0062 - val_loss: 7.9801 - val_accuracy: 0.0057
Epoch 6/1000
33/33 [==============================] - 0s 5ms/step - loss: 6.4756 - accuracy: 0.0143 - val_loss: 8.1395 - val_accuracy: 0.0095
Epoch 7/1000
33/33 [==============================] - 0s 5ms/step - loss: 6.2808 - accuracy: 0.0162 - val_loss: 8.2749 - val_accuracy: 0.0095
E